In [46]:
from functools import lru_cache
import time
import pandas as pd
from pandas import DataFrame
import pandasql
import cufflinks as cf
cf.go_offline()
cf.set_config_file(theme='henanigans', colorscale='henanigans')

from shekels.core.database import Database
import shekels.core.data_tools as sdt

In [6]:
@lru_cache()
def foo(data):
    time.sleep(1)
    return data

# d = DataFrame()
# d['val'] = [5, 4]
# d = str(d)
d = 'foo'
# foo(d)
info = foo.cache_info()
# info.index(d)

In [30]:
regex = pp.Regex('~')
not_regex = pp.Regex('!~')
select = pp.Regex('select', flags=re.I).setResultsName('select')
from_ = pp.Suppress(pp.Regex('from', flags=re.I))
table = (from_ + pp.Regex('[a-z]+', flags=re.I)).setParseAction(lambda s, l, t: t[0]).setResultsName('table')
any_ = pp.Regex('any', flags=re.I).setResultsName('any')
all_ = pp.Regex('all', flags=re.I).setResultsName('all')
and_ = pp.Regex('and', flags=re.I).setResultsName('and')
or_ = pp.Regex('or', flags=re.I).setResultsName('or')
where = pp.Suppress(pp.Regex('where', flags=re.I))
like = pp.Regex('like', flags=re.I)
is_ = pp.Regex('is', flags=re.I)
not_ = pp.Regex('not', flags=re.I)
eq = pp.Regex('==', flags=re.I)
neq = pp.Regex('!=', flags=re.I)
gt = pp.Regex('>', flags=re.I)
gte = pp.Regex('>=', flags=re.I)
lt = pp.Regex('<', flags=re.I)
lte = pp.Regex('<=', flags=re.I)
operator = pp.Or([like, is_, eq, neq, gt, gte, lt, lte, regex, not_regex]).setResultsName('operator')
quote = pp.Suppress(pp.Optional("'"))
value = (quote + pp.Regex('[^\']+', flags=re.I) + quote) \
    .setResultsName('value') \
    .setParseAction(lambda s, l, t: t[0])
columns = pp.Regex('\*|[a-z]+', flags=re.I).setResultsName('display_columns')
column = pp.Regex('[a-z]+', flags=re.I).setResultsName('column')
conditional = column + operator + value
head = select + columns + table + where + conditional
conn = and_ | or_
# query = head + pp.ZeroOrMore(conn + query)
query = head + conditional + conn + conditional

text = 'select 8651267 from data where foo ~ bar'
text = 'select * from data where foo ~ bar'
text = "select * from data where foo ~ 'bar baz'"
text = 'select * from data where foo ~ bar and food == bagel'
# output = query.parseString(text).asDict()
output = query.parseString(text).asDict()
output
# output = None
# try:
#     output = query.parseString(text).asDict()
# except pp.ParseException:
#     output = text
# output

ParseException: Expected Re:('[a-z]+') (at char 52), (line:1, col:53)

In [2]:
config = '/root/shekels/resources/config.json'
data = Database.from_json(config).update().data

In [13]:
def get_mintapi_data(source):
    with open(source) as f:
        data = json.load(f)

    lut = dict(
        odate='date',
        merchant='description',
        omerchant='original_description',
        amount='amount',
        account='account',
        category='category',
        labels='labels',
        note='notes',
        fi='financial_institution'
    )
    data = DataFrame(data)
    del data['date']
    data.rename(
        lambda x: lut[x] if x in lut.keys() else x,
        axis=1,
        inplace=True
    )
    data.rename(lbt.as_snakecase, axis=1, inplace=True)
    data.date = data.date.apply(
        lambda x: datetime.utcfromtimestamp(int(str(x)[:-3]))
    )
#     data = data[lut.values()]
    return data


#     lut = get_periodicity_table(data, 'date', 'description', 'amount')
def get_periodicity_table(data, group_column, key_column, value_column):
    data = data.copy()
    data[group_column] = data[group_column].apply(
        lambda x: x.replace(microsecond=randint(0, 1000000))
    )
    cols = [group_column, key_column, value_column]
    data = data[cols].pivot(*cols)
    data[group_column] = data.index
    data.reset_index(drop=True, inplace=True)

    score = DataFrame()
    date_intervals = [
        'year',
        'half_year',
        'quarter_year',
        'month',
        'half_month',
        'week',
        'day',
        'half_day',
    ]
    for period in date_intervals:
        prob = data.copy()
        prob[group_column] = prob[group_column].apply(lambda x: conform_date(x, period))
        prob = prob.groupby(group_column).count()
        cnt = prob.sum()
        prob = prob.applymap(lambda x: 0 if x == 0 else 1.0 / x)
        prob = prob.mean() * cnt
        score[period] = prob

    score = score.T
    output = DataFrame()
    output[key_column] = score.columns.tolist()
    output['period'] = score.apply(lambda x: np.argmax(x)).tolist()
    output['score'] = score.apply(lambda x: np.max(x)).tolist()
    output.sort_values('score', ascending=False, inplace=True)
    output.reset_index(drop=True, inplace=True)
    return output


def date_pivot(data, key_column, value_column):
    data.date = data.date.apply(lambda x: x.replace(microsecond=randint(0, 1000000)))
    cols = ['date', key_column, value_column]
    data = data[cols].pivot(*cols)
    data['date'] = data.index.tolist()
    data.date = data.date.apply(lambda x: x.replace(microsecond=0))
    data.reset_index(drop=True, inplace=True)
    return data


def conform_date(date, period):
    if period == 'year':
        return datetime(date.year, 1, 1)

    if period == 'half_year':
        if date.month >= 7:
            return datetime(date.year, 7, 1)
        return datetime(date.year, 1, 1)

    if period == 'quarter_year':
        if date.month >= 10:
            return datetime(date.year, 10, 1)
        elif date.month >= 7:
            return datetime(date.year, 7, 1)
        elif date.month < 4:
            return datetime(date.year, 4, 1)
        else:
            return datetime(date.year, 1, 1)

    if period == 'month':
        return datetime(date.year, date.month, 1)

    if period == 'half_month':
        if date.day >= 16:
            return datetime(date.year, date.month, 16)
        return datetime(date.year, date.month, 1)

    if period == 'week':
        if date.day >= 22:
            return datetime(date.year, date.month, 22)
        elif date.day >= 15:
            return datetime(date.year, date.month, 15)
        elif date.day >= 8:
            return datetime(date.year, date.month, 8)
        else:
            return datetime(date.year, date.month, 1)

    if period == 'day':
        return datetime(date.year, date.month, date.day)

    if period == 'half_day':
        if date.hour >= 13:
            return datetime(date.year, date.month, date.day, 13)
        return datetime(date.year, date.month, date.day, 1)

    raise ValueError(period + ' is not a valid period')
